**4.	Trends für verschiedenen Kryptowährungen vergleichen, Ereignisse (z.B politische Ereignisse)**

In [36]:
from dotenv import load_dotenv
import requests
from api_key_coingecko import API_KEY
import numpy as np
import pandas as pd
from openai import OpenAI
import os
import json
import textwrap


In [37]:
load_dotenv()

True

In [38]:
# The role for openai prompt
role = """Du bist ein Krypto-Marktanalyse-Bot.

Antwortsprache: **Deutsch**.

Aufgabe:
1. Suche mithilfe von Web-Recherche nach relevanten Ereignissen, die den genannten Kurs­sprung/-fall erklären könnten.
2. Gib **exakt** folgendes JSON-Array zurück (keine Kommentare, kein Markdown):

[
  {
    "zeitraum": "<TT.MM.JJJJ-TT.MM.JJJJ>",
    "preisänderung": "<+12.34 %>",
    "ursache": "<max. 2 Sätze, keine URLs>",
    "ereignistyp": "<Politik | Markt | Regulierung | Meme/Trend | Unbekannt>",
    "vertrauen": "<Hoch | Mittel | Gering>",
    "quellen": ["<mind. 1, max. 5 valide URLs>"]
  }
]

Richtlinien:
- Nenne **mindestens eine** Quelle; falls keine verlässliche Quelle gefunden, setze leeres Array und `vertrauen="Gering"`.
- Füge **keine** URL in `ursache` ein.
- Nutze Social-Media-Erwähnungen (X/Twitter, Reddit) für Meme-Coins, kennzeichne das als `ereignistyp="Meme/Trend"`."""


In [39]:
#connecting coingecko api to extryct coin prices (can be deleted after merge to avoid redundancy)
url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart" #api end point 

days_to_analyse = 2
currency = 'usd'

headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": API_KEY
}

params = {
    'vs_currency' : currency,
    'days' : str(days_to_analyse)
}

response = requests.get(url, params=params, headers=headers)
data = response.json()

In [40]:
# the same func to load token prices (can be deleted after merge to avoid redundancy)
response.raise_for_status()
def load_token_prices(days : int, token:str, currency = 'usd',):
    try : 
        url = f"https://api.coingecko.com/api/v3/coins/{token}/market_chart"
        
        params = {
            'vs_currency': currency,
            'days': days
        }
        response = requests.get(url, params=params, headers=headers)
        data = response.json()

        prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
        prices['timestamp'] = pd.to_datetime(prices['timestamp'], unit='ms')
        prices.set_index('timestamp', inplace=True)

        return prices
    except:
        return 'An exception occured.'

In [41]:
"""
    Die Funktion liefert die stärksten prozentualen Kursbewegungen (auf- oder abwärts) 
    eines Tokens über rollierende Zeitfenster.

    • window: Länge des Fensters in Tagen/Zeilen  
    • direction: 'up', 'down' oder 'both'  
    • n_results: maximale Zahl der zurückgegebenen, nicht überlappenden Fenster  
    • allow_overlap=False verhindert, dass sich Zeiträume überschneiden

    Rückgabe: DataFrame mit Start- und Enddatum, Preisen, %-Änderung und Richtung.
"""
def biggest_moves(prices, window=5, n_results=None, direction="both", allow_overlap=False):
    if "price" not in prices:
        raise ValueError("prices DataFrame must contain column 'price'")

    # Calculate % change
    pct_change = prices["price"].pct_change(periods=window) * 100

    # Create the DataFrame
    df = pd.DataFrame({
        "end_price": prices["price"],
        "pct_change": pct_change
    }).dropna()

    df["start_price"] = df["end_price"].shift(window)
    df["start"] = df.index - pd.Timedelta(days=window)
    df["end"] = df.index
    df["direction"] = np.where(df["pct_change"] > 0, "up", "down")
    df["abs_move"] = df["pct_change"].abs()

    # Filter by direction
    if direction in {"up", "down"}:
        df = df[df["direction"] == direction]

    # Sort by absolute move
    ranked = df.sort_values("abs_move", ascending=False)

    # Remove overlapping periods
    if not allow_overlap:
        kept_idx, spans = [], []
        for idx, row in ranked.iterrows():
            s, e = row["start"], row["end"]
            if not any((s <= ee) and (e >= ss) for ss, ee in spans):
                kept_idx.append(idx)
                spans.append((s, e))
            if n_results is not None and len(kept_idx) == n_results:
                break
        ranked = ranked.loc[kept_idx]

    # Final output
    result = (
        ranked[["start", "end", "start_price", "end_price", "pct_change", "direction"]]
        .sort_values("pct_change", ascending=(direction == "down"))
        .head(n_results)
        .reset_index(drop=True)
        .round({"start_price": 2, "end_price": 2, "pct_change": 2})
    )
     # Strip time from datetime
    result["start"] = pd.to_datetime(result["start"]).dt.date
    result["end"] = pd.to_datetime(result["end"]).dt.date
    
    return result

In [42]:
""" Die Funktion ruft openai api auf und rescheschiert, ob die Kursbewegungen mit globalen Ereignissen zur Verbindung stehen.
  Und Wenn ja, aus welchen Gründen Tokenpreis an- oder abgestiegen ist. """
  
def load_news(token, start, end, pct_change):

  client = OpenAI(
      api_key = os.getenv('OPENAI_API_KEY')
      )

  message = f'{token}, period: from  {start} to {end}, returns: {pct_change}'

  response = client.chat.completions.create(
    model="gpt-4o-search-preview",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "text": role,
            "type": "text"
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    web_search_options={
      "user_location": {
        "type": "approximate",
        "approximate": {
          "country": "DE"
        }
      }
    }
  )
  return response.choices[0].message.content

In [48]:
token = 'shiba'

prices = load_token_prices(days=365,token=token)
result = biggest_moves(prices, window=7, n_results=3)
print(result)
json_str = load_news(token=token, start=result['start'], end= result['end'], pct_change=result['pct_change'])
events = json.loads(json_str)
print(events)

for event in events:
    print(f"\nZeitraum: {event['zeitraum']}")
    print(f"Preisänderung: {event['preisänderung']}")
    print(textwrap.fill(event['ursache'], width=80))
    print(f"Ereignistyp: {event['ereignistyp']}")
    print(f"Vertrauen: {event['vertrauen']}")
    print("Quellen:")
    if event['quellen']: 
        for url in event['quellen']:
            print(f"  - {url}")
    else: print('Keine Quellen')
    print()

        start         end  start_price  end_price  pct_change direction
0  2025-03-22  2025-03-29          0.0        0.0       85.85        up
1  2024-09-16  2024-09-23          0.0        0.0       72.42        up
2  2024-06-16  2024-06-23          NaN        0.0      -62.95      down
[{'zeitraum': '22.03.2025-29.03.2025', 'preisänderung': '+85.85 %', 'ursache': 'Ein technischer Ausbruch über signifikante Widerstandsniveaus führte zu einem starken Aufwärtsmomentum.', 'ereignistyp': 'Markt', 'vertrauen': 'Mittel', 'quellen': ['https://coinedition.com/de/der-preisanstieg-von-shiba-inu-im-jahr-2025-eine-aufwaertsprognose-von-264/']}, {'zeitraum': '16.09.2024-23.09.2024', 'preisänderung': '+72.42 %', 'ursache': 'Die Ankündigung neuer Stablecoins im Shiba Inu-Ökosystem führte zu erhöhter Nachfrage und Preissteigerungen.', 'ereignistyp': 'Markt', 'vertrauen': 'Mittel', 'quellen': ['https://cryptonews.net/de/news/altcoins/29864305/']}, {'zeitraum': '16.06.2024-23.06.2024', 'preisänderung': 